# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import *
import subprocess, base64
from pathlib import Path
from PIL import Image
from gaspard import *
import uuid, asyncio, shutil

app, rt = fast_app()

task_status = {}

@rt("/animation.gif")
async def get_gif(): return FileResponse("animation.gif")

@rt("/sessions/{task_id}/animation.gif")
async def get_session_gif(task_id: str): return FileResponse(f"./data/sessions/{task_id}/animation.gif")

@rt("/status/{task_id}")
async def get_status(task_id: str):
    task = task_status.get(task_id, dict(progress=0, status='not_found', step='Task not found'))
    
    if task['status'] == 'complete': 
        return Div(
            P("✅ Animation complete!"), 
            Img(src=f"/data/sessions/{task_id}/animation.gif", style="max-width:100%; border-radius:8px; box-shadow:0 4px 8px rgba(0,0,0,0.1)")
        )
    elif task['status'] == 'not_found':
        return Div(P("❌ Task not found"))
    else:
        return Div(
            P(f"🔄 {task['step']}"), 
            Progress(value=task['progress'], max=100, style="width:100%; margin:10px 0"), 
            P(f"Progress: {task['progress']}%", style="font-size:0.9em; color:#666"),
            id="progress", 
            hx_get=f"/status/{task_id}", 
            hx_trigger="every 2s", 
            hx_swap="outerHTML"
        )

@rt
async def index():
    return Titled("Draw something to help the stickman reach the star!",
        Div(
            Div(
                Button("Red", onclick="setColor('red')", style="background:red;color:white;margin:2px"),
                Button("Blue", onclick="setColor('blue')", style="background:blue;color:white;margin:2px"),
                Button("Black", onclick="setColor('black')", style="background:black;color:white;margin:2px"),
            ),
            Canvas(id="canvas", width="1024", height="1024", style="border:1px solid black"),
            Br(),
            Button("Save Drawing", onclick="submitCanvas()", id="submit-btn"),
            Div(id="result", style="margin-top:20px"),

            Script("""
                const canvas = document.getElementById('canvas');
                const ctx = canvas.getContext('2d');
                let drawing = false;
                let currentColor = 'black';

                const img = new Image();
                img.onload = function() { ctx.drawImage(img, 0, 0, canvas.width, canvas.height); };
                img.src = '/data/frame0.png';

                function setColor(color) { currentColor = color; }

                canvas.addEventListener('mousedown', e => { drawing = true; draw(e); });
                canvas.addEventListener('mousemove', draw);
                canvas.addEventListener('mouseup', () => { drawing = false; ctx.beginPath(); });

                function draw(e) {
                    if (!drawing) return;
                    ctx.lineWidth = 3; ctx.lineCap = 'round'; ctx.strokeStyle = currentColor;
                    ctx.lineTo(e.clientX - canvas.offsetLeft, e.clientY - canvas.offsetTop);
                    ctx.stroke(); ctx.beginPath();
                    ctx.moveTo(e.clientX - canvas.offsetLeft, e.clientY - canvas.offsetTop);
                }

                function submitCanvas() {
                    const dataURL = canvas.toDataURL('image/png');
                    htmx.ajax('POST', '/submit_drawing', {values: {canvas_data: dataURL}, target: '#result'});
                }
            """)
        )
    )

async def create_animation_async(task_id: str, canvas_data: str):
    task_status[task_id] = dict(progress=0, status='running', step='Initializing animation...')
    
    session_dir = Path(f'./data/sessions/{task_id}')
    session_dir.mkdir(parents=True, exist_ok=True)
    shutil.copy(Path('data/frame0.png'), session_dir / 'frame0.png')

    task_status[task_id].update(progress=10, step='Setting up session...')
    
    img_data = canvas_data.split(',')[1]
    img_bytes = base64.b64decode(img_data)
    with open(session_dir / 'canvas_drawing.png', 'wb') as f: f.write(img_bytes)
    
    task_status[task_id].update(progress=20, step='Analyzing drawing...')
    
    chat = Chat("gemini-2.5-flash-image-preview")
    chat([session_dir / "frame0.png", session_dir / "canvas_drawing.png", """\
You're evaluating what the user drew on the original image and will animate it into a gif.
First decide if what the user drew should enable the character reach the star.
If the drawing of the user is not helpful the character should fall in the pit or fly off screen or just now shrug. Fails should be funny!
          
Second, you describe what the succesfull or unsuccesful action looks like. You ALWAYS do so in 6 steps.
Describe in 6 frames how the user's addition enables the character to reach the star or how they fail (one of these NOT both). 
Describe each frame in a single sentence and return them as a numbered list. 
We will generate those frames and combine them into a gif. 

It's important you don't add any other attributes besides what the user has drawn, but you can make creative use of it and add some pzazz. 
For example, if the character gets a trampoline instead of just jumping you can describe a fancy flip. 

It's important that you do not modify the original landscape.
"""])
    
    for i in range(1, 7):
        progress = 20 + (i-1) * 10  # Start at 20, increment by 10 for each frame
        task_status[task_id].update(progress=progress, step=f'Generating frame {i}/6...')
        
        res = chat(f"Ok now generate frame {i}")
        with open(session_dir / f'frame{i}.png', 'wb') as f: f.write(res.candidates[0].content.parts[0].inline_data.data)
        
        progress_after = 20 + i * 10
        task_status[task_id].update(progress=progress_after, step=f'Frame {i}/6 completed')
    
    task_status[task_id].update(progress=90, step='Creating animation...')
    
    img0 = Image.open(session_dir / 'frame0.png').resize((1024, 1024))
    img0.save(session_dir / 'frame0_resized.png')
    
    img0 = Image.open(session_dir / 'canvas_drawing.png').resize((1024, 1024))
    img0.save(session_dir / 'canvas_drawing_resized.png')
    
    subprocess.run(['convert', '-delay', '20', str(session_dir / 'frame0_resized.png'), str(session_dir / 'canvas_drawing_resized.png')] + [str(session_dir / f'frame{i}.png') for i in range(1, 7)] + [str(session_dir / 'animation.gif')])
    
    task_status[task_id] = dict(progress=100, status='complete', step='Animation complete!', gif_path=str(session_dir / 'animation.gif'))

@rt("/submit_drawing", methods=["POST"])
async def submit_drawing(canvas_data: str):
    task_id = str(uuid.uuid4())
    asyncio.create_task(create_animation_async(task_id, canvas_data))
    return Div(P("🔄 Animation started..."), Progress(value=0, max=100), id="progress", hx_get=f"/status/{task_id}", hx_trigger="every 2s", hx_swap="outerHTML")